# Importing Libraries

In [1]:
# EDA
import pandas as pd
import numpy as np

# Data Preprocessing 
from sklearn import preprocessing

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Recommender System Imps
# Content Based Filtering 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Collaborative Based Filtering 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# To work with text data 
import re
import string

# Importing Data 

In [2]:
df = pd.read_csv('./Book1.csv',encoding = 'latin-1')
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,Spicy Green Papaya Salad,Thai,veg,"Som Tam is a flavorful, green papaya salad dis..."
1,2,Thai Fried Noodles,Thai,veg,ad Thai is one of the most iconic stir-fried T...
2,3,Hamburger,American,non-veg,A sandwich made with a beef patty and various ...
3,4,Raw Papaya Salad,Thai,veg,This Som Tum salad is as delicious as it is fi...
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


In [3]:
# No of dishes in my dataset
len(list(df['Name'].unique()))

289

In [4]:
df['C_Type'].unique() # Categorical Data 

array(['Thai', 'American', 'Dessert', 'Chinese', 'Indian', 'Italian',
       'Beverage'], dtype=object)

In [5]:
df['Veg_Non'].unique() # Categorical Data 

array(['veg', 'non-veg'], dtype=object)

In [6]:
len(df) 
# Hmm... Small Dataset!

309

In [7]:
df.shape

(309, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Food_ID   309 non-null    int64 
 1   Name      309 non-null    object
 2   C_Type    309 non-null    object
 3   Veg_Non   309 non-null    object
 4   Describe  309 non-null    object
dtypes: int64(1), object(4)
memory usage: 12.2+ KB


In [9]:
# Let's make a function to remove all the punctuation from the "Describe" column
def text_cleaning(text):
    text  = "".join([char for char in text if char not in string.punctuation])    
    return text

In [10]:
# Let's clean the text 
df['Describe'] = df['Describe'].apply(text_cleaning)

In [11]:
# Let's see if that worked...
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,Spicy Green Papaya Salad,Thai,veg,Som Tam is a flavorful green papaya salad dish...
1,2,Thai Fried Noodles,Thai,veg,ad Thai is one of the most iconic stirfried Th...
2,3,Hamburger,American,non-veg,A sandwich made with a beef patty and various ...
3,4,Raw Papaya Salad,Thai,veg,This Som Tum salad is as delicious as it is fi...
4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...


In [12]:
# Are there any duplicate data ?
df.duplicated().sum()
# None :)

0

In [13]:
# Are there any null values?
df.isnull().sum()
# None :))

Food_ID     0
Name        0
C_Type      0
Veg_Non     0
Describe    0
dtype: int64

In [14]:
# General Description 
df.describe()

,Food_ID
count,309.000000
mean,155.000000
std,89.344838
min,1.000000
25%,78.000000
50%,155.000000
75%,232.000000
max,309.000000


# Content Based Filtering 

# 1. Simple Content Based Filtering

### What is TF-IDF?
In a large text corpus, some words will be very present (e.g. “the”, “a”, “is” in English) hence carrying very little meaningful information about the actual contents of the document. If we were to feed the direct count data directly to a classifier those very frequent terms would shadow the frequencies of rarer yet more interesting terms.

In order to re-weight the count features into floating point values suitable for usage by a classifier it is very common to use the tf–idf transform.

Thanks to https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [15]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Describe'])
tfidf_matrix.shape

(309, 967)

### What is Linear Kernel ?
The function linear_kernel computes the linear kernel, that is, a special case of polynomial_kernel with degree=1 and coef0=0 (homogeneous). 

Thanks to https://scikit-learn.org/stable/modules/metrics.html#linear-kernel

In [16]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.15850277, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15850277, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [17]:
# Just considering the Food names from the dataframe
indices = pd.Series(df.index, index=df['Name']).drop_duplicates()
indices

Name
Spicy Green Papaya Salad      0
Thai Fried Noodles            1
Hamburger                     2
Raw Papaya Salad              3
christmas cake                4
                           ... 
Veg Hakka Noodles           304
Pecan pie                   305
Dosa                        306
Double ka meetha            307
Banana pudding              308
Length: 309, dtype: int64

In [18]:
# The main recommender code!
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar food
    sim_scores = sim_scores[1:6]
    
    food_indices = [i[0] for i in sim_scores]
    return df['Name'].iloc[food_indices]

# 2. Advanced Content Based Filtering

In [19]:
# Including all features that will help in recommending better
features = ['C_Type','Veg_Non', 'Describe']

In [20]:
# Soup represents a mixture of elements 
# Similarly, I am making one column that will have all the important features 
# I am simply concatenating the strings 

def create_soup(x):
    return x['C_Type'] + " " + x['Veg_Non'] + " " + x['Describe']

In [21]:
# Using the soup(), I am creating the column for the dataframe df
df['soup'] = df.apply(create_soup, axis=1)

In [22]:
# Checking out if that worked!
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe,soup
0,1,Spicy Green Papaya Salad,Thai,veg,Som Tam is a flavorful green papaya salad dish...,Thai veg Som Tam is a flavorful green papaya s...
1,2,Thai Fried Noodles,Thai,veg,ad Thai is one of the most iconic stirfried Th...,Thai veg ad Thai is one of the most iconic sti...
2,3,Hamburger,American,non-veg,A sandwich made with a beef patty and various ...,American non-veg A sandwich made with a beef p...
3,4,Raw Papaya Salad,Thai,veg,This Som Tum salad is as delicious as it is fi...,Thai veg This Som Tum salad is as delicious as...
4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...,Dessert veg christmas dry fruits presoaked ora...


### What is Count Vectorizer ?
Convert a collection of text documents to a matrix of token counts. This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

### What is fit_transform ?
Learn the vocabulary dictionary and return document-term matrix.

Thanks to https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [23]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

### What is Cosine Similarity? 
The function computes cosine similarity between samples in X and Y.

Thanks to https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

cosine_similarity computes the L2-normalized dot product of vectors. This is called cosine similarity, because Euclidean (L2) normalization projects the vectors onto the unit sphere, and their dot product is then the cosine of the angle between the points denoted by the vectors.

Thanks to https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity

In [24]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [25]:
# Reseting the index and pulling out the names of the food alone from the df dataframe
df = df.reset_index()
indices = pd.Series(df.index, index=df['Name'])

In [26]:
# Let's see the names of the food pulled out
display(indices)

Name
Spicy Green Papaya Salad      0
Thai Fried Noodles            1
Hamburger                     2
Raw Papaya Salad              3
christmas cake                4
                           ... 
Veg Hakka Noodles           304
Pecan pie                   305
Dosa                        306
Double ka meetha            307
Banana pudding              308
Length: 309, dtype: int64

# Testing Content Based Filtering

In [27]:
# This is the first model - simple variation 
get_recommendations('Dosa')

177              Idli
163         Idiyappam
95          Daal puri
216    Keerai masiyal
283           Payasam
Name: Name, dtype: object

In [28]:
# This is the second model - advanced variation 
get_recommendations('Dosa', cosine_sim2)

177               Idli
163          Idiyappam
214              Kanji
246    Kothamali sadam
281      Paruppu sadam
Name: Name, dtype: object

# Collaborative Filtering 

In [29]:
# Importing the ratings file
rating = pd.read_csv('ratings.csv')
rating.head()

,User_ID,Food_ID,Rating
0,1,88,4
1,1,46,5
2,1,24,4
3,1,25,5
4,2,49,5


In [30]:
#Checking the shape
rating.shape

(763, 3)

In [31]:
# Checking for null values 
rating.isnull().sum()

User_ID    0
Food_ID    0
Rating     0
dtype: int64

In [32]:
# I actually saw the data earlier and found that the last row had no values 
# Let's see 
rating.tail()

,User_ID,Food_ID,Rating
758,348,36,3
759,349,130,4
760,350,96,5
761,351,41,5
762,352,34,5


In [33]:
# Removing the last row 
rating = rating[:511]
rating.tail()

,User_ID,Food_ID,Rating
506,99,65,4
507,99,22,1
508,100,24,5
509,100,233,5
510,100,29,4


In [34]:
# So, now there should not be any null value 
rating.isnull().sum()

User_ID    0
Food_ID    0
Rating     0
dtype: int64

In [35]:
# Making a dataframe that has food ID and the number of ratings
food_rating = rating.groupby(by = 'Food_ID').count()
food_rating = food_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
food_rating

,Food_ID,Rating_count
0,1,2
1,2,3
2,3,2
3,4,2
4,5,6
...,...,...
304,305,1
305,306,1
306,307,1
307,308,1


In [36]:
# Food rating dataframe description
food_rating['Rating_count'].describe()

count    309.000000
mean       1.653722
std        1.107748
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        7.000000
Name: Rating_count, dtype: float64

In [37]:
# The user rating dataframe shows the number of ratings given with respect to the user
user_rating = rating.groupby(by='User_ID').count()
user_rating = user_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
user_rating

,User_ID,Rating_count
0,1,4
1,2,4
2,3,9
3,4,6
4,5,6
...,...,...
95,96,6
96,97,7
97,98,7
98,99,6


In [38]:
# User rating dataframe description 
user_rating["Rating_count"].describe()

count    100.000000
mean       5.110000
std        2.352282
min        1.000000
25%        3.000000
50%        5.000000
75%        7.000000
max       11.000000
Name: Rating_count, dtype: float64

In [39]:
# Ultimate Table
rating_matrix = rating.pivot_table(index='Food_ID',columns='User_ID',values='Rating').fillna(0)
rating_matrix.head()

User_ID,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
Food_ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0


In [40]:
# Shape of rating_matrix
rating_matrix.shape

(309, 100)

### What is csr_matrix?
Sparse matrix is a matrix that contains a lot of zeros. This functions helps to compress the sparse matrix. 
Thanks to https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [41]:
csr_rating_matrix =  csr_matrix(rating_matrix.values)
print(csr_rating_matrix)

  (0, 48)	5.0
  (0, 70)	5.0
  (1, 8)	4.0
  (1, 21)	5.0
  (1, 38)	5.0
  (2, 76)	1.0
  (2, 88)	4.0
  (3, 40)	3.0
  (3, 89)	3.0
  (4, 27)	5.0
  (4, 38)	5.0
  (4, 72)	4.0
  (4, 87)	3.0
  (4, 93)	2.0
  (4, 97)	4.0
  (5, 42)	5.0
  (5, 48)	3.0
  (5, 76)	3.0
  (5, 91)	1.0
  (6, 16)	4.0
  (6, 47)	5.0
  (6, 62)	5.0
  (6, 69)	4.0
  (6, 70)	4.0
  (7, 4)	4.0
  :	:
  (284, 30)	5.0
  (285, 80)	3.0
  (286, 24)	3.0
  (287, 54)	3.0
  (288, 55)	5.0
  (289, 31)	4.0
  (290, 15)	5.0
  (291, 2)	3.0
  (292, 95)	5.0
  (293, 41)	4.0
  (294, 43)	5.0
  (295, 41)	5.0
  (296, 94)	5.0
  (297, 55)	4.0
  (298, 2)	5.0
  (299, 28)	5.0
  (300, 53)	1.0
  (301, 77)	5.0
  (302, 63)	3.0
  (303, 29)	1.0
  (304, 55)	5.0
  (305, 79)	4.0
  (306, 70)	1.0
  (307, 96)	3.0
  (308, 31)	5.0


### What is Nearest Neighbours?
Reference Link: https://scikit-learn.org/stable/modules/neighbors.html

In [42]:
# Using cosine similarity to find nearest neigbours 
recommender = NearestNeighbors(metric='cosine')
recommender.fit(csr_rating_matrix)

NearestNeighbors(metric='cosine')

In [43]:
# The main recommender code!
def Get_Recommendations(title):
    user= df[df['Name']==title]
    user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]
    user_ratings = rating_matrix.iloc[user_index]

    reshaped = user_ratings.values.reshape(1,-1)
    distances, indices = recommender.kneighbors(reshaped,n_neighbors=16)
    
    nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]
    nearest_neighbors = pd.DataFrame({'Food_ID': nearest_neighbors_indices})
    
    result = pd.merge(nearest_neighbors,df,on='Food_ID',how='left')
    
    return result.head()

In [44]:
# Get recommendations with this function 
Get_Recommendations('Dosa')

,Food_ID,index,Name,C_Type,Veg_Non,Describe,soup
0,307,306,Dosa,Indian,veg,PancakeHopper Ground rice urad dal,Indian veg PancakeHopper Ground rice urad dal
1,1,0,Spicy Green Papaya Salad,Thai,veg,Som Tam is a flavorful green papaya salad dish...,Thai veg Som Tam is a flavorful green papaya s...
2,50,49,Panzanella,Italian,veg,a bread salad with tomatoes onions and basil,Italian veg a bread salad with tomatoes onions...
3,7,6,chocolate nero cookies,Dessert,veg,almonds eggs granulated sugar bittersweet choc...,Dessert veg almonds eggs granulated sugar bitt...
4,53,52,Cheeseburger,American,non-veg,A hamburger topped with cheese,American non-veg A hamburger topped with cheese


# Upvote if you like this ✨